# Pre-Pre-Processing

Before I start performing any EDA or modeling, I will pre-emptively split my data to have a separate training set of data.

Since the data consists of temporal data (hotel reservations across multiple years), I will split the data into two groups based on two datetime features: Booking Date and the Arrival date.

---

# Consulting ChatGPT

I asked ChatGPT to help me decide if one feature would be better than the other. It's responses are as follows:

## Booking Date

### Pros:
> * *Forecasting Lead Time:* Splitting by booking date allows you to model the lead time between booking and arrival, which is valuable for forecasting future demand well in advance.
> * *Real-world Scenario Modeling:* This approach mirrors real-world scenarios more closely, where predictions are made as bookings occur, regardless of the future arrival date.
> * *Dynamic Pricing and Revenue Management:* Particularly useful for applications in dynamic pricing and revenue management, where understanding booking patterns ahead of time can inform strategic decisions.

### Cons:
> * *Increased Complexity:* The time gap between booking and arrival dates introduces additional complexity, as you need to account for changes over time that could affect the stay (e.g., cancellations, modifications).



## Arrival Date

### Pros:
> * *Cleaner Data Relationships:* Utilizing the arrival date might result in cleaner, more straightforward modeling, as it directly ties the prediction to the period of service or product consumption.
> * *Seasonality and Trend Analysis:* Easier to model seasonal trends and effects, as the arrival date directly corresponds to the period being analyzed.
> * *Accuracy in Performance Metrics:* Predictions based on arrival date can be more closely aligned with actual occupancy and revenue, potentially improving model accuracy in terms of performance metrics.

### Cons:

> * *Reduced Forecasting Horizon:* The model may be less effective at predicting bookings well in advance since it's oriented around the arrival date. This could limit its usefulness for long-term planning.
> * *Possible Lag in Actionable Insights:* May not provide as much lead time for implementing strategies based on the predictions, such as staffing or promotional offers, since the focus is on the period closer to the actual stay.

---

# Consultation Conclusions

After considering ChatGPT's suggestions and insights, I will take a more greedy approach and create separate datasets for both the booking and arrival dates. This will give me more flexibility when modeling as I will have different time perspectives to utilize for different purposes (e.g., future forecasting vs. analyzing actualized performance).

---

# Date Preparation

Before I can split the datasets, I need to perform some slight feature engineering. The source datasets do not have an exact datetime feature for the arrival date, only for the booking date. I will use the separate Year, Month, and Day of Month features to create an `Arrival_Date` feature, then use this feature for splitting my data.

In [10]:
from bmc_functions.db_utils import duckdb_connection
import duckdb
import pandas as pd


In [2]:
### OLD DATA - Retaining for debugging

# ## Maintaing separate hotel data

# hotel_number = '1'
# # hotel_number = '2'

# path = f'./data/H{hotel_number}.parquet'
# df_data = pd.read_parquet(path)

# df_data.head()

# Read Data from DuckDB

In [11]:
# Path to the DuckDB database file
db_path = './data/hotel_reservations.duckdb'

with duckdb_connection(db_path) as conn:
    df_data = conn.execute("SELECT * FROM source_data").df()

display(df_data.head(10))

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,HotelNumber,UUID
0,0,342,2015,July,27,1,0,0,2,0.0,...,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01,1,6f4f201b-62de-4c33-b9da-15081cf7e359
1,0,737,2015,July,27,1,0,0,2,0.0,...,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01,1,ad3d542c-41eb-4957-9724-32d3638d6bec
2,0,7,2015,July,27,1,0,1,1,0.0,...,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02,1,6a0684e0-9157-4456-9a53-1685d506a951
3,0,13,2015,July,27,1,0,1,1,0.0,...,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02,1,512c5081-3be1-4f78-87dc-2b0c8a9e9bde
4,0,14,2015,July,27,1,0,2,2,0.0,...,NULL,0,Transient,98.0,0,1,Check-Out,2015-07-03,1,444ea515-6e6b-45b4-9870-fcf46b5b529c
5,0,14,2015,July,27,1,0,2,2,0.0,...,NULL,0,Transient,98.0,0,1,Check-Out,2015-07-03,1,5c55ead3-f96e-44b6-b167-f0f0c87e013c
6,0,0,2015,July,27,1,0,2,2,0.0,...,NULL,0,Transient,107.0,0,0,Check-Out,2015-07-03,1,ba6dccfe-4082-4ce8-86dd-9fc130c84d3c
7,0,9,2015,July,27,1,0,2,2,0.0,...,NULL,0,Transient,103.0,0,1,Check-Out,2015-07-03,1,ea4d5557-6767-458a-8f3d-929cea401bc5
8,1,85,2015,July,27,1,0,3,2,0.0,...,NULL,0,Transient,82.0,0,1,Canceled,2015-05-06,1,d5ef1deb-f28a-45bb-b7bc-aac85dad3425
9,1,75,2015,July,27,1,0,3,2,0.0,...,NULL,0,Transient,105.5,0,0,Canceled,2015-04-22,1,f07c2799-6be5-4a8d-93a5-ee099cfa3e3c


In [6]:
## Convert Arrival columns to strings

arrival_date_cols = ['ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateDayOfMonth']

arrival_date_cols_str = df_data[arrival_date_cols].astype(str)
arrival_date_cols_str.head()

,ArrivalDateYear,ArrivalDateMonth,ArrivalDateDayOfMonth
0,2015,July,1
1,2015,July,1
2,2015,July,1
3,2015,July,1
4,2015,July,1


In [7]:
## Create new column of strings formatted as YYYY-MM-DD, then convert to datetime

arrival_date_full_str = arrival_date_cols_str['ArrivalDateYear'] + '-' + \
                        arrival_date_cols_str['ArrivalDateMonth'] + '-' + \
                        arrival_date_cols_str['ArrivalDateDayOfMonth']

arrival_date_dt = pd.to_datetime(arrival_date_full_str, yearfirst = True)
arrival_date_dt.name = 'Arrival_Date'
arrival_date_dt

0        2015-07-01
1        2015-07-01
2        2015-07-01
3        2015-07-01
4        2015-07-01
            ...    
119385   2017-08-30
119386   2017-08-31
119387   2017-08-31
119388   2017-08-31
119389   2017-08-29
Name: Arrival_Date, Length: 119390, dtype: datetime64[ns]

In [8]:
## Concatenate new column
df_data = pd.concat([df_data, arrival_date_dt], axis = 1)
df_data.head()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,HotelNumber,UUID,Arrival_Date
0,0,342,2015,July,27,1,0,0,2,0.0,...,0,Transient,0.0,0,0,Check-Out,2015-07-01,1,ae05b69e-fcce-4b16-99d8-cf26faf9c4e6,2015-07-01
1,0,737,2015,July,27,1,0,0,2,0.0,...,0,Transient,0.0,0,0,Check-Out,2015-07-01,1,473a8d2f-6d16-41f1-a22f-e77f2fea5b7a,2015-07-01
2,0,7,2015,July,27,1,0,1,1,0.0,...,0,Transient,75.0,0,0,Check-Out,2015-07-02,1,7ff91b07-290f-4061-a823-377c47aadc08,2015-07-01
3,0,13,2015,July,27,1,0,1,1,0.0,...,0,Transient,75.0,0,0,Check-Out,2015-07-02,1,3342411f-c149-4d0a-a400-ac21e757eb21,2015-07-01
4,0,14,2015,July,27,1,0,2,2,0.0,...,0,Transient,98.0,0,1,Check-Out,2015-07-03,1,6e81bc1a-c90b-46d5-b90c-d3509dc939c8,2015-07-01


In [6]:
df_data = df_data.drop(columns=[ 'ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateDayOfMonth'])
df_data

,IsCanceled,LeadTime,ArrivalDateWeekNumber,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,...,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,Arrival_Date
0,0,342,27,0,0,2,0,0,BB,PRT,...,NULL,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01,2015-07-01
1,0,737,27,0,0,2,0,0,BB,PRT,...,NULL,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01,2015-07-01
2,0,7,27,0,1,1,0,0,BB,GBR,...,NULL,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02,2015-07-01
3,0,13,27,0,1,1,0,0,BB,GBR,...,304,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02,2015-07-01
4,0,14,27,0,2,2,0,0,BB,GBR,...,240,NULL,0,Transient,98.00,0,1,Check-Out,2015-07-03,2015-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40055,0,212,35,2,8,2,1,0,BB,GBR,...,143,NULL,0,Transient,89.75,0,0,Check-Out,2017-09-10,2017-08-31
40056,0,169,35,2,9,2,0,0,BB,IRL,...,250,NULL,0,Transient-Party,202.27,0,1,Check-Out,2017-09-10,2017-08-30
40057,0,204,35,4,10,2,0,0,BB,IRL,...,250,NULL,0,Transient,153.57,0,3,Check-Out,2017-09-12,2017-08-29
40058,0,211,35,4,10,2,0,0,HB,GBR,...,40,NULL,0,Contract,112.80,0,1,Check-Out,2017-09-14,2017-08-31


In [7]:
## Create timedelta series based on number of weekday/end nights.
timedelta_wknd = pd.to_timedelta(df_data.loc[:, 'StaysInWeekendNights'], unit = 'D')
timedelta_wk = pd.to_timedelta(df_data.loc[:, 'StaysInWeekNights'], unit = 'D')

In [8]:
## Calculate the departure date by adding the timedeltas to the arrival date
departure_date = df_data.loc[:, 'Arrival_Date'] + timedelta_wk + timedelta_wknd
departure_date.name = 'Departure_Date'
departure_date

0       2015-07-01
1       2015-07-01
2       2015-07-02
3       2015-07-02
4       2015-07-03
           ...    
40055   2017-09-10
40056   2017-09-10
40057   2017-09-12
40058   2017-09-14
40059   2017-09-14
Name: Departure_Date, Length: 40060, dtype: datetime64[ns]

In [9]:
## Concatenate with original dataframe
df_data = pd.concat([df_data, departure_date], axis = 1)
df_data.head()

,IsCanceled,LeadTime,ArrivalDateWeekNumber,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,...,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,Arrival_Date,Departure_Date
0,0,342,27,0,0,2,0,0,BB,PRT,...,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01,2015-07-01
1,0,737,27,0,0,2,0,0,BB,PRT,...,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01,2015-07-01
2,0,7,27,0,1,1,0,0,BB,GBR,...,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,2015-07-02
3,0,13,27,0,1,1,0,0,BB,GBR,...,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,2015-07-02
4,0,14,27,0,2,2,0,0,BB,GBR,...,NULL,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01,2015-07-03


In [10]:
df_data = df_data.drop(columns=['StaysInWeekendNights', 'StaysInWeekNights'])
df_data

,IsCanceled,LeadTime,ArrivalDateWeekNumber,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,...,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,Arrival_Date,Departure_Date
0,0,342,27,2,0,0,BB,PRT,Direct,Direct,...,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01,2015-07-01,2015-07-01
1,0,737,27,2,0,0,BB,PRT,Direct,Direct,...,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01,2015-07-01,2015-07-01
2,0,7,27,1,0,0,BB,GBR,Direct,Direct,...,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02,2015-07-01,2015-07-02
3,0,13,27,1,0,0,BB,GBR,Corporate,Corporate,...,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02,2015-07-01,2015-07-02
4,0,14,27,2,0,0,BB,GBR,Online TA,TA/TO,...,NULL,0,Transient,98.00,0,1,Check-Out,2015-07-03,2015-07-01,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40055,0,212,35,2,1,0,BB,GBR,Offline TA/TO,TA/TO,...,NULL,0,Transient,89.75,0,0,Check-Out,2017-09-10,2017-08-31,2017-09-10
40056,0,169,35,2,0,0,BB,IRL,Direct,Direct,...,NULL,0,Transient-Party,202.27,0,1,Check-Out,2017-09-10,2017-08-30,2017-09-10
40057,0,204,35,2,0,0,BB,IRL,Direct,Direct,...,NULL,0,Transient,153.57,0,3,Check-Out,2017-09-12,2017-08-29,2017-09-12
40058,0,211,35,2,0,0,HB,GBR,Offline TA/TO,TA/TO,...,NULL,0,Contract,112.80,0,1,Check-Out,2017-09-14,2017-08-31,2017-09-14


In [11]:
leadtime_timedelta = pd.to_timedelta(df_data['LeadTime'], unit = 'D')
leadtime_timedelta

0       342 days
1       737 days
2         7 days
3        13 days
4        14 days
          ...   
40055   212 days
40056   169 days
40057   204 days
40058   211 days
40059   161 days
Name: LeadTime, Length: 40060, dtype: timedelta64[ns]

In [12]:
df_data['Booking_Date'] = df_data['Arrival_Date'] - leadtime_timedelta
df_data['Booking_Date']

0       2014-07-24
1       2013-06-24
2       2015-06-24
3       2015-06-18
4       2015-06-17
           ...    
40055   2017-01-31
40056   2017-03-14
40057   2017-02-06
40058   2017-02-01
40059   2017-03-23
Name: Booking_Date, Length: 40060, dtype: datetime64[ns]

In [13]:
# df_data = df_data.drop(columns = ['LeadTime']) ## Maintain for future modeling

df_data.head(10)

,IsCanceled,LeadTime,ArrivalDateWeekNumber,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,...,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,Arrival_Date,Departure_Date,Booking_Date
0,0,342,27,2,0,0,BB,PRT,Direct,Direct,...,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01,2015-07-01,2014-07-24
1,0,737,27,2,0,0,BB,PRT,Direct,Direct,...,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01,2015-07-01,2013-06-24
2,0,7,27,1,0,0,BB,GBR,Direct,Direct,...,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,2015-07-02,2015-06-24
3,0,13,27,1,0,0,BB,GBR,Corporate,Corporate,...,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,2015-07-02,2015-06-18
4,0,14,27,2,0,0,BB,GBR,Online TA,TA/TO,...,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01,2015-07-03,2015-06-17
5,0,14,27,2,0,0,BB,GBR,Online TA,TA/TO,...,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01,2015-07-03,2015-06-17
6,0,0,27,2,0,0,BB,PRT,Direct,Direct,...,0,Transient,107.0,0,0,Check-Out,2015-07-03,2015-07-01,2015-07-03,2015-07-01
7,0,9,27,2,0,0,FB,PRT,Direct,Direct,...,0,Transient,103.0,0,1,Check-Out,2015-07-03,2015-07-01,2015-07-03,2015-06-22
8,1,85,27,2,0,0,BB,PRT,Online TA,TA/TO,...,0,Transient,82.0,0,1,Canceled,2015-05-06,2015-07-01,2015-07-04,2015-04-07
9,1,75,27,2,0,0,HB,PRT,Offline TA/TO,TA/TO,...,0,Transient,105.5,0,0,Canceled,2015-04-22,2015-07-01,2015-07-04,2015-04-17


In [14]:
df_data['Arrival_Date'].max() - df_data['Arrival_Date'].min()

Timedelta('792 days 00:00:00')

In [15]:
df_data['Booking_Date'].max() - df_data['Booking_Date'].min()

Timedelta('1529 days 00:00:00')

# Subset with 10% Threshold for Arrival_Date and Booking_Date

Despite the sizable number of rows for each hotel's dataset, the Arrival_Date feature is limited in scope - covering a little over 2 years' worth of data. This limited time range restricts the possible forecasting windows, particularly as I intend to reserve a certain number of days for the validation sets.

My compromise between availability and integrity is to subset each hotel's separate datasets by 10% of the total dataset. This will preserve the size of the original data's timeframe while providing a reasonable time window for future forecasting.

In [16]:
def split_and_save_dataset_by_percentage(df, date_column, hotel_number, percentage=10, save_path='./data'):
    """
    Split the dataset into training and holdout sets based on a specified percentage of the total rows
    and save them. The holdout set will contain the last 'percentage' of rows based on the datetime column.
    """
    
    # Work on a copy of the DataFrame to preserve the original data
    df_copy = df.copy()
    
    # Ensure the date column is in datetime format and set it as the index
    df_copy[date_column] = pd.to_datetime(df_copy[date_column])
    df_copy = df_copy.sort_values(by=[date_column])

    # Calculate the index to split on: last 'percentage' of rows
    split_idx = int(len(df_copy) * (1 - (percentage / 100)))

    # Split the dataset and reset index for future use
    train_df = df_copy.iloc[:split_idx, :]
    train_df = train_df.reset_index(drop=True)
    
    holdout_df = df_copy.iloc[split_idx:, :]
    holdout_df = holdout_df.reset_index(drop=True)

    # No need to reset index if you want to keep the datetime column in its place
    # Define file paths
    training_path = f'{save_path}/H{hotel_number}_Training.parquet'
    holdout_path = f'{save_path}/H{hotel_number}_Validation.parquet'

    # Save the datasets
    train_df.to_parquet(training_path, engine='pyarrow', compression='brotli')
    holdout_df.to_parquet(holdout_path, engine='pyarrow', compression='brotli')


In [17]:
date_columns = ['Arrival_Date', 'Booking_Date']

for date_column in date_columns:
    split_and_save_dataset_by_percentage(df_data, date_column, hotel_number, 10, f'./data/Datasets_for_{date_column}')


# Results

After running this code for both of the hotel datasets, I now have separate training and validation datasets for each of the hotels' Arrival_Date and Booking_Date features. The resulting datasets will be effective for modeling and forecasting in later notebooks.